In [1]:
import qualified Data.Sequence       as Seq
import qualified Data.Set            as Set
import qualified Data.Map.Strict     as M
import Data.List

input <- readFile "../input/day24.txt"

mapMap :: String -> M.Map (Int, Int) Char
mapMap input = M.fromList [((x,y), c) | (y, line) <- zip [0..] (lines input), (x, c) <- zip [0..] line, c /='#']


findPos p maze = let
    list = M.toList maze
    Just (coord,_) = find (\v -> snd v ==p) list
    in coord

neighbours (x,y) maze visited = let
    possible = [(x-1,y), (x+1, y), (x,y-1), (x,y+1)]
    paths    = filter (\c -> M.member c maze && Set.notMember c visited) possible
    in paths

to coords maze queue visited = case Seq.viewl queue of
    (coord, count) Seq.:< t | coord == coords -> count
    (coord, count) Seq.:< t                   -> let
        visited' = Set.insert coord visited
        ns       = neighbours coord maze visited
        count'   = count + 1
        queue'   = (Seq.dropWhileL (\(c,_) -> Set.member c visited') t) Seq.>< Seq.fromList (map (\n -> (n, count')) ns)
        in to coords maze queue' visited'

shortestPath  start dest maze = to dest maze (Seq.singleton (start, 0)) Set.empty
shortestPath' start dest maze = shortestPath (findPos start maze) (findPos dest maze)  maze


mz = mapMap input

In [2]:
test = unlines
    [ "###########"
    , "#0.1.....2#"
    , "#.#######.#"
    , "#4.......3#"
    , "###########"
    ]

testInput = mapMap test

shortestPath' '0' '4' testInput
shortestPath' '0' '4' mz

2

238

In [3]:
paths = [[x,y] | x <- "01234567", y <- "01234567", y /= x]
    
pathMap = foldl' (\acc [a,b] -> M.insert [a,b] (shortestPath' a b mz) acc) M.empty paths

In [4]:
possible = map ('0':) $ permutations "1234567"

cost costs path = let
    entries = zipWith (\a b -> [a,b]) path (tail path)
    in sum $ map (\pair -> costs M.! pair) entries

minimum $ map (cost pathMap) possible

474

In [5]:
possible' = map (++"0") possible

minimum $ map (cost pathMap) possible'

696